# IBDB Webscraping
This file webscrapes publically available data from the Broadway League's IBDB (Internet Broadway Database) for the purposes of analyzing the importance of Tony Awards outcomes for Broadway Productions.

This file borrows code in its first few chunks and takes general inspiration from the following [Colaboratory Jupyter Notebook](https://colab.research.google.com/drive/1IVwOhBMYay14NkO7kGkrPu0Ij9dSDdEP) by Yaakov Bressler.

In [1]:
import io
import requests
from lxml import etree
from bs4 import BeautifulSoup
import urllib.request
import re
import string
#import time
#import json   #might not need commented out ones
import datetime
import pandas as pd
import numpy as np
import urllib
import ast
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### Create a function that grabs links from a page, using a tag to identify value of link

In [2]:
def getLinks_tagged_fast(url, tag):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    links = []
    # set the opening of each link to be...
    tag = tag
    for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
        links.append(link.get('href'))
    return links

https://www.broadwayworld.com/browseshows.cfm?showtype=BR

The above link is your starting point. It will allow us to get the name of every Broadway production that opened between 1979 and now.

In [3]:
def get_show_links_year(year_url):
    """
    This function...
    """
    url = year_url
    tag_year = 'browseshows.cfm?'
    #calling previous function to get the link I want
    years = getLinks_tagged_fast(url, tag_year)[1:]
    page_base = 'https://www.broadwayworld.com/'
    years_loop =[]
    for year in years:
        #focusing on 1979 or later
        if year[-4:].isdigit() and int(year[-4:]) >= 1979:
            years_loop.append(page_base+year)
    
    # Now you have all the years
    tag_show = 'https://www.broadwayworld.com/shows/backstage.php?'
    show_links_nested = []
    for year in years_loop:
        show_links_nested.append(getLinks_tagged_fast(year,tag_show))
    show_links = sum(show_links_nested, [])
    
    return show_links
    
#running function to get my list of links to the productions I want
year_url = 'https://www.broadwayworld.com/browseshows.cfm?showtype=BR'
show_links = get_show_links_year(year_url)

In [4]:
#don't need every show ever, just 1979 onwards
#from here, you will the scrape name of each show, so you can search for it on ibdb.com using selenium
#print(show_links)
#the number of productions between 1979 and now fits the estimate I made in another assignment (roughly 1800)
len(show_links)

1877

In [5]:
def get_show_name(url):
    """
    This function...
    """
    response = requests.get(url)
    #make sure url actually exists
    assert response.status_code == 200

    show_html = response.text
    htmlparser = etree.HTMLParser()
    tree = etree.parse(io.StringIO(show_html), parser=htmlparser)
    showroot = tree.getroot()
    #this xml path is the same for every url, it will give you the name of the show for that url
    show_name = showroot.xpath("//span[@itemprop = 'name']/text()")[1]
    
    return show_name

In [6]:
#testing the function on a random url (tried this with several urls)
get_show_name('https://www.broadwayworld.com/shows/backstage.php?showid=6366')

'Oliver!'

In [7]:
#using get_show_name function to make a list of every single show I want to find data on
#took about 10 minutes to run this chunk
show_names = []
for link in show_links:
    show_name = get_show_name(link)
    show_names.append(show_name)
    
len(show_names)

1877

In [8]:
#due to the presence of revivals in the list, some show names appear more than once
#therefore, to reduce redundancy when I search for these shows on the database, I am only keeping unique show names
unique_show_names = []
for show in show_names:
    if show not in unique_show_names:
        unique_show_names.append(show)

#print(unique_show_names)     
len(unique_show_names)

1590

In [12]:
#show_names

In [9]:
#the xpath statement that will get me the href for each production I am interested in
prod_xpath = '//div[@id = "nyc-productions" and @data-id = "nyc-productions"]/div/div/div/div/a'#/@href'

#testing functions from selenium package on three shows before doing every show

#this will be a list all of the urls that I need 
urls_list = []
driver = webdriver.Chrome()
productions = ['The King and I', 'Dear Evan Hansen', 'Spring Awakening'] #add 'Rodgers + Hammerstein's Cinderella'
for prod in productions:
    driver.get('https://www.ibdb.com/shows/')     #website I am searching from (IBDB)
    search_box = driver.find_element('name','ShowProperName')    #locating the searchbar
    search_box.send_keys(prod)       #automating the searches (will store results in a list later)
    search_box.submit()
    #search_box.send_keys(Keys.ENTER)  #other way to do above line of code
    web_elts = driver.find_elements('xpath', prod_xpath)
    #getting href I need from each selenium WebElement
    results = [elt.get_attribute("href") for elt in web_elts]
    urls_list.extend(results)


In [10]:
urls_list#[0][41:]

['https://www.ibdb.com/broadway-production/the-king-and-i-1935',
 'https://www.ibdb.com/broadway-production/the-king-and-i-3999',
 'https://www.ibdb.com/broadway-production/the-king-and-i-4357',
 'https://www.ibdb.com/broadway-production/the-king-and-i-4673',
 'https://www.ibdb.com/broadway-production/the-king-and-i-497593',
 'https://www.ibdb.com/broadway-production/dear-evan-hansen-508238',
 'https://www.ibdb.com/broadway-production/spring-awakening-448811',
 'https://www.ibdb.com/broadway-production/spring-awakening-501403']

In [47]:
#Must do this for every single production I search for, just doing small handful now

#this LoD (poorly structured because of IBDB layout) will have weekly gross/capacity info
prod_data_list = []

#this LoD will have Tony nom/win info
tony_data_LoD = []

#this xpath give me the name of each award the production was nominated for
noms_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div/div/div/h4/text()"
#this xpath will give me the number of awards the production won
wins_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div[@class='col s1 right-align']/img[@src = '/Images/award.png']"
#this xpath will give me the year that the production was eligible for awards
#this path only works if the production received noms, for productions with 0 noms, I might have to manually put in the year
year_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div[@class = 'col s11']/div/div[@class = 'col s12' and position() = 2]/text()"


for url in urls_list:
    resp = requests.get(url)
    prod_html = resp.text
    soup = BeautifulSoup(prod_html, 'html.parser')
    #finding node that has javascript text with our data
    script = soup.find_all('script', type='text/javascript')[1]   #node at [1] is one with our data
    #making script into a string so I can easily parse through it
    js: str = script.text
        
    #need a 'try:' because the next few lines of code will not work for productions with no finanical data 
    #(i.e. pre 1979)
    try:
        #using a regex to search for the dict we are looking for (i.e. the one that has the data)
        raw_json = re.search('var grossdata = {0:\[.*\] };', js, flags=re.MULTILINE).group(0)
        #[16:-1] to exclude javascript syntax stuff
        data = ast.literal_eval(raw_json[16:-1])
        #adding key:value pair to dict to keep track of which production is which
        data['production'] = url[41:]
        prod_data_list.append(data)
        
        #scraping Tony info now
        #We must do this within the try because this allows us to skip Tony info for productions with no financial data
        #REACH GOAL: split awards into major/minor categories
        # ^ Based on the layout of the website, it would be tedious/difficult to do this
        tree = etree.HTML(str(soup)) 
        nominations = tree.xpath(noms_path)
        num_noms = len(nominations)
        num_wins = len(tree.xpath(wins_path))
        year = tree.xpath(year_path)[0]
        prod_award_dict = {'production': url[41:], 'nominations': num_noms, 'wins': num_wins, 'year': int(year[26:30])}
        tony_data_LoD.append(prod_award_dict)
    except:
        pass

In [48]:
tony_data_LoD

[{'production': 'the-king-and-i-4357',
  'nominations': 3,
  'wins': 1,
  'year': 1985},
 {'production': 'the-king-and-i-4673',
  'nominations': 8,
  'wins': 4,
  'year': 1996},
 {'production': 'the-king-and-i-497593',
  'nominations': 9,
  'wins': 4,
  'year': 2015},
 {'production': 'dear-evan-hansen-508238',
  'nominations': 9,
  'wins': 6,
  'year': 2017},
 {'production': 'spring-awakening-448811',
  'nominations': 11,
  'wins': 8,
  'year': 2007},
 {'production': 'spring-awakening-501403',
  'nominations': 3,
  'wins': 0,
  'year': 2016}]

In [13]:
prod_data_list[0]

{0: [['Jun 9, 1985',
   '$407,169',
   '76%',
   '9,969',
   '71%',
   'Jun 9',
   407169.0,
   534619.0,
   9969.0,
   14120.0,
   '0',
   '8'],
  ['Jun 16, 1985',
   '$177,896',
   '67%',
   '4,644',
   '66%',
   'Jun 16',
   177896.0,
   267309.5,
   4644.0,
   7060.0,
   '0',
   '4'],
  ['Jun 23, 1985',
   '$318,174',
   '95%',
   '8,025',
   '91%',
   'Jun 23',
   318174.0,
   334136.87,
   8025.0,
   8825.0,
   '0',
   '5'],
  ['Jun 30, 1985',
   '$605,546',
   '113%',
   '13,833',
   '98%',
   'Jun 30',
   605546.0,
   534619.0,
   13833.0,
   14120.0,
   '0',
   '8']],
 1: [['Dec 30, 1984',
   '$482,280',
   '90%',
   '11,832',
   '84%',
   'Dec 30',
   482280.0,
   534619.0,
   11832.0,
   14120.0,
   '8',
   '0'],
  ['Jan 6, 1985',
   '$502,566',
   '94%',
   '13,255',
   '94%',
   'Jan 6',
   502566.0,
   534619.0,
   13255.0,
   14120.0,
   '8',
   '0'],
  ['Jan 13, 1985',
   '$465,216',
   '87%',
   '13,652',
   '97%',
   'Jan 13',
   465216.0,
   534619.0,
   13652.0,
   

In [62]:
#TO DO: Make these comments (and others) into markdown chunks
#structure of prod_data_list (it is a poorly structured LoD):
#each dict is one production
#each key represent one season of data, with the exception of the key I added to represent the production
#the value for each season key is an LoL 
#each list in the LoL represents one week of data for that production
#in each list, we care about the vals at indeces: 0 (date of week), 4 (weekly capacity), 6 (weekly gross) 

#end result should be LoD of the following structure:
#[{'production': 'the-king-and-i-497593', 'date': 'May 29, 2016', 'capacity':'76%', 'gross': 546476.0},...]

prod_LoD = []
#prod_data_list[2][0][0]
for prod in prod_data_list:
    for season in prod:
        for week in prod[season]:
            if type(week) == list:
                #print(week)
                relevant_data = {'production': prod['production'], 'date': week[0], 'capacity': int(week[4][:-1]), 'gross': week[6]}
                prod_LoD.append(relevant_data)

#see what first few rows of data look like
prod_LoD[:5]

[{'production': 'the-king-and-i-4357',
  'date': 'Jun 9, 1985',
  'capacity': 71,
  'gross': 407169.0},
 {'production': 'the-king-and-i-4357',
  'date': 'Jun 16, 1985',
  'capacity': 66,
  'gross': 177896.0},
 {'production': 'the-king-and-i-4357',
  'date': 'Jun 23, 1985',
  'capacity': 91,
  'gross': 318174.0},
 {'production': 'the-king-and-i-4357',
  'date': 'Jun 30, 1985',
  'capacity': 98,
  'gross': 605546.0},
 {'production': 'the-king-and-i-4357',
  'date': 'Dec 30, 1984',
  'capacity': 84,
  'gross': 482280.0}]

In [49]:
#TO DO: DELETE THIS CHUNK

#this xpath give me the name of each award the production was nominated for
#noms_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div/div/div/h4/text()"
#this xpath will give me the number of awards the production won
#wins_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div[@class='col s1 right-align']/img[@src = '/Images/award.png']"
#tony_data_list = []

#for url in urls_list:
#test = 'https://www.ibdb.com/broadway-production/the-king-and-i-497593'
#resp = requests.get(test)
#prod_html = resp.text
#soup = BeautifulSoup(prod_html, 'html.parser')
#tree = etree.HTML(str(soup)) 
#year = tree.xpath("//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div[@class = 'col s11']/div/div[@class = 'col s12' and position() = 2]/text()")[0]
#nominations = tree.xpath(noms_path)
#num_noms = len(nominations)
#num_wins = len(tree.xpath(wins_path))
#prod_award_dict = {"production": url[41:], "nominations": num_noms, "wins": num_wins}
#tony_data_list.append(prod_award_dict)

In [76]:
#tony_data_list

In [39]:
#print(nominations)
#num_wins

In [65]:
#TO DO: should also scrape year of ceremony for each of these
tony_data = pd.DataFrame(tony_data_LoD)
tony_data = tony_data.set_index('production')
tony_data

,nominations,wins,year
production,,,
the-king-and-i-4357,3,1,1985
the-king-and-i-4673,8,4,1996
the-king-and-i-497593,9,4,2015
dear-evan-hansen-508238,9,6,2017
spring-awakening-448811,11,8,2007
spring-awakening-501403,3,0,2016


In [69]:
#TO DO: remove '%' symbol from capacity
weekly_data = pd.DataFrame(prod_LoD)
weekly_data = weekly_data.set_index(['production','date'])
weekly_data

capacity     gross
production              date                            
the-king-and-i-4357     Jun 9, 1985         71  407169.0
                        Jun 16, 1985        66  177896.0
                        Jun 23, 1985        91  318174.0
                        Jun 30, 1985        98  605546.0
                        Dec 30, 1984        84  482280.0
...                                        ...       ...
spring-awakening-501403 Dec 27, 2015        78  467253.0
                        Jan 3, 2016         87  688989.0
                        Jan 10, 2016        93  618339.0
                        Jan 17, 2016        93  604880.0
                        Jan 24, 2016        98  545145.0

[541 rows x 2 columns]

In [92]:
#save to data folder
#tony_data.to_csv('data/tony_data.csv')
#weekly_data.to_csv('data/weekly_data.csv')